In [ ]:
# 包引入和全局变量定义模块
import pandas as pd
from jqdata import *
import seaborn as sns
from datetime import datetime,timedelta,date
import numpy as np
import matplotlib.pyplot as plt
import json
import math
import random
from pyfinance import TSeries as tss
from functools import reduce


sns.set_style("whitegrid",{"font.sans-serif":['simhei','Droid Sans Fallback']})
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)



In [ ]:
def util_get_stock_sw_cate(list_stock: list, date_q=(datetime.today() - timedelta(1)).date()):
    l_code = []
    l_sw_1_code = []
    l_sw_1_name = []
    l_sw_2_code = []
    l_sw_2_name = []
    l_sw_3_code = []
    l_sw_3_name = []
    dict_industry_data = get_industry(security=list_stock)
    for code, value in dict_industry_data.items():
        l_code.append(code)
        l_sw_1_code.append(value['sw_l1']['industry_code'] if value.get('sw_l1') is not None else '')
        l_sw_2_code.append(value['sw_l2']['industry_code'] if value.get('sw_l2') is not None else '')
        l_sw_3_code.append(value['sw_l3']['industry_code'] if value.get('sw_l3') is not None else '')
        l_sw_1_name.append(value['sw_l1']['industry_name'] if value.get('sw_l1') is not None else '')
        l_sw_2_name.append(value['sw_l2']['industry_name'] if value.get('sw_l2') is not None else '')
        l_sw_3_name.append(value['sw_l3']['industry_name'] if value.get('sw_l3') is not None else '')
    df_industry = pd.DataFrame.from_dict(
        {'code': l_code,
         'sw_1_code': l_sw_1_code, 'sw_1_name': l_sw_1_name,
         'sw_2_code': l_sw_2_code, 'sw_2_name': l_sw_2_name,
         'sw_3_code': l_sw_3_code, 'sw_3_name': l_sw_3_name})
    return df_industry


def util_get_fund_portfolio_by_chunks(code_list: list, start_date, end_date, rec_type):
    """
    在start_date 到 end_date 日期内，list_stock里的股票被所有基金的持仓情况统计
    """

    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    pd_result = pd.DataFrame.from_dict({
        'symbol': [], 'start_date': [], 'end_date': [], 'report_type': [], 'stock_count': [],
        'share_sum': [], 'market_cap_sum': []
    })

    for once_code in chunks(code_list, 60):

        q = query(finance.FUND_PORTFOLIO_STOCK).filter(finance.FUND_PORTFOLIO_STOCK.code.in_(once_code),
                                                       finance.FUND_PORTFOLIO_STOCK.pub_date <= end_date,
                                                       finance.FUND_PORTFOLIO_STOCK.pub_date >= start_date,
                                                       finance.FUND_PORTFOLIO_STOCK.report_type == rec_type,
                                                       finance.FUND_PORTFOLIO_STOCK.rank <= 50,
                                                       )

        fund_portfolio = finance.run_query(q)

        stock_als = fund_portfolio.groupby(['symbol', 'report_type']).agg(
            {'shares': 'sum', 'symbol': 'count', 'market_cap': 'sum'}).rename(
            columns={'symbol': 'stock_count', 'shares': 'share_sum', 'market_cap': 'market_cap_sum'}).reset_index()

        if len(stock_als) != 0:
            stock_als['start_date'] = start_date
            stock_als['end_date'] = end_date
            stock_als = stock_als[
                ['symbol', 'start_date', 'end_date', 'report_type', 'stock_count', 'share_sum', 'market_cap_sum']]
            pd_result = pd.concat([pd_result, stock_als]).groupby(
                ['symbol', 'start_date', 'end_date', 'report_type']).sum().reset_index()

    return pd_result


def util_normalize_code(pd_data, code_col_name='code'):
    """
    通过股票code 得到股票名称
    """
    l_code_o = pd_data[code_col_name].values.tolist()
    l_norm_code = []

    for one_code in l_code_o:
        try:
            l_norm_code.append(normalize_code(one_code))
        except:
            l_norm_code.append('unknown')

    pd_data['code'] = l_norm_code
    return pd_data


def util_run_func_by_chunk(list_code, n, func, **args):
    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    result = []

    for once_code in chunks(list_code, n):
        result.append(func(once_code, **args))

    return result


def util_add_name(pd_data, code_col_name='code'):
    """
    通过股票code 得到股票名称
    """

    l_code_o = pd_data[code_col_name].values.tolist()
    l_name = []

    for one_code in l_code_o:
        try:
            l_name.append(get_security_info(one_code).display_name)
        except:
            l_name.append('未知')

    pd_data['display_name'] = l_name
    return pd_data


def util_add_pub_year(pd_data, code_col_name='code'):
    """
    通过股票code 得到股票名称
    """

    l_code_o = pd_data[code_col_name].values.tolist()
    l_name = []

    for one_code in l_code_o:
        try:
            l_name.append(get_security_info(one_code).start_date)
        except:
            l_name.append('未知')

    pd_data['pub_date'] = l_name
    return pd_data


def util_get_month_K(list_code):
    close = history(security_list=list_code, unit='1d', count=5 * 365, field='close')
    close = close.resample('M').last().unstack()

    open = history(security_list=list_code, unit='1d', count=5 * 365, field='open')
    open = open.resample('M').first().unstack()

    high = history(security_list=list_code, unit='1d', count=5 * 365, field='high')
    high = high.resample('M').max().unstack()

    low = history(security_list=list_code, unit='1d', count=5 * 365, field='low')
    low = low.resample('M').min().unstack()

    result = pd.concat([open, close, high, low], axis=1).reset_index().rename(
        columns={'level_0': 'code', 'level_1': 'end_date', 0: 'open', 1: 'close', 2: 'high', 3: 'low'})
    result['end_date'] = result['end_date'].map(lambda x: x.strftime('%Y-%m-%d'))
    return result


def util_drop_unnanme_col(data):
    col_names = data.columns
    drop_cols = []
    for col in col_names:
        if col.find('Unnamed:') >= 0:
            drop_cols.append(col)
    return data.drop(columns=drop_cols)


def util_get_month_paused(list_code):
    paused = history(security_list=list_code, unit='1d', count=5 * 365, field='paused')
    paused = paused.resample('M').last().unstack().reset_index().rename(
        columns={'level_0': 'code', 'level_1': 'end_date', 0: 'paused'})
    paused['end_date'] = paused['end_date'].map(lambda x: x.strftime('%Y-%m-%d'))
    return paused


def get_K_raise_3m_later(list_code, start_date):
    all_month_d = util_get_month_K(list_code)
    all_month_d['start_date'] = start_date
    all_month_d = all_month_d[all_month_d['end_date'] >= start_date]
    all_month_d = all_month_d.groupby(['code']).head(4)
    all_month_d['month_K_raise_rate'] = all_month_d['close'].pct_change()
    all_month_d = all_month_d.pivot_table(index=['code', 'start_date'], columns=['end_date'],
                                          values='month_K_raise_rate')
    all_month_d.columns = ['mk_rate_' + one_col for one_col in all_month_d.columns]
    return all_month_d.reset_index()


def get_pd_K_raise_3m_later(pd_data):
    result = []
    l_code = list(set(pd_data['code'].values.tolist()))
    all_stock_result = util_get_month_K(l_code)
    all_stock_result['month_K_raise_rate'] = all_stock_result.groupby(['code'])['close'].pct_change()
    all_stock_result = all_stock_result.dropna()
    all_stock_result = all_stock_result.sort_values(by=['code', 'end_date'])
    l_end_date = list(set(pd_data['end_date'].values.tolist()))

    result_tmp = []
    for start_date in l_end_date:
        all_month_d = all_stock_result[all_stock_result['end_date'] >= start_date]
        all_month_d_get_index = all_stock_result[all_stock_result['end_date'] == start_date][['code']]
        all_month_d = pd.merge(all_month_d, all_month_d_get_index, on=['code'])
        all_month_d = all_month_d.sort_values(by=['code', 'end_date']).groupby(['code']).head(4)
        all_month_d['start_date'] = start_date
        all_month_d = all_month_d.pivot_table(index=['code', 'start_date'], columns=['end_date'],
                                              values='month_K_raise_rate')
        all_month_d.columns = ['mk_rate_' + str(i) for i in range(len(all_month_d.columns.tolist()))]
        result_tmp.append(all_month_d.reset_index())

    pd_result = pd.concat(result_tmp, ignore_index=True).reset_index()
    result_final = pd.merge(pd_data, pd_result, left_on=['code', 'end_date'], right_on=['code', 'start_date'])
    del result_final['start_date']
    return result_final


def util_get_month_turnOver(list_code):
    def util_get_month_turnOver_inner(list_code_tmp):
        list_result_tmp = util_run_func_by_chunk(list_code_tmp, 5, get_valuation, end_date='2021-03-12', count=365 * 5,
                                                 fields=['turnover_ratio'])
        pd_result = pd.concat(list_result_tmp)
        pd_result['end_date'] = pd.to_datetime(pd_result['day'], format='%Y-%m-%d')
        del pd_result['day']
        pd_result = pd.pivot_table(pd_result, index=['end_date'], columns=['code'], values=['turnover_ratio']).fillna(
            -1)
        pd_result = pd_result.resample('M').mean()
        pd_result.columns = pd_result.columns.levels[1]
        pd_result = pd_result.unstack().reset_index().rename(columns={0: 'turnOver_monthly'})
        pd_result['end_date'] = pd_result['end_date'].map(lambda x: x.strftime('%Y-%m-%d'))
        return pd_result

    def chunks(lst, n=10):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    result = []

    for once_code in chunks(list_code):
        result.append(util_get_month_turnOver_inner(once_code))

    return pd.concat(result)


def util_get_all_month(list_year, to_str=True):
    result = []

    def last_day_of_month(any_day):
        next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
        return next_month - datetime.timedelta(days=next_month.day)

    for year in list_year:
        for month in range(1, 13):
            day = last_day_of_month(datetime.date(year, month, 1))
            if to_str:
                result.append(day.strftime('%Y-%m-%d'))
            else:
                result.append(day)
    return result


def util_buy_stocks(money_available, end_date, stocks, pd_weight=None):
    len_stocks = len(stocks)
    if len_stocks == 0:
        return None, 0
    stock_price = get_price(stocks, count=1, end_date=end_date, frequency='daily', fields=['close'], panel=False)
    if pd_weight is None:
        stock_price['money'] = money_available / len_stocks
    elif type(pd_weight) == list:
        stock_price['money'] = [int(money_available * weight) for weight in pd_weight]
    else:
        pd_weight = pd_weight[['code', 'weight']].drop_duplicates()
        if len(pd_weight) != len(stock_price):
            raise ValueError()
        stock_price = pd.merge(stock_price, pd_weight, on=['code'])
        stock_price['money'] = stock_price.apply(lambda x: int(x['weight'] * money_available), axis=1)
    stock_price['stock_num'] = stock_price.apply(lambda x: int((x['money'] / x['close']) / 100) * 100, axis=1)
    stock_price = stock_price.rename(columns={'close': 'buy_price'})
    stock_price['buy_date'] = end_date

    stock_price = stock_price[['code', 'buy_price', 'buy_date', 'stock_num']]

    stock_price['buy_money'] = stock_price.apply(lambda x: x['stock_num'] * x['buy_price'], axis=1)
    return stock_price, stock_price['buy_money'].sum()



def util_sell_stocks(data_for_sell, one_date):

    if len(data_for_sell)==0:
        return None,None,0


    list_code = data_for_sell['code'].values.tolist()

    stock_price = get_price(list_code, count=1, end_date=one_date, frequency='daily',
                            fields=['paused', 'close'], panel=False)[['code', 'paused', 'close']]
    stock_price.rename(columns={'close': 'sell_price'}, inplace=True)

    data_result = pd.merge(data_for_sell, stock_price, on=['code'])
    data_result['sell_date'] = one_date
    data_result_sell = data_result[data_result['paused'] == 0]
    data_result_not_sell = data_result[data_result['paused'] == 1]
    sell_money = 0
    if len(data_result_sell) > 0:
        data_result_sell['profit'] = data_result_sell.apply(
            lambda x: (x['sell_price'] - x['buy_price']) * x['stock_num'], axis=1)

        data_result_sell['sell_money'] = data_result_sell.apply(
            lambda x: x['sell_price'] * x['stock_num'], axis=1)

        sell_money = data_result_sell['sell_money'].sum()
    else:
        data_result_sell = None

    if len(data_result_not_sell) > 0:
        data_result_not_sell = data_result_not_sell[['code', 'buy_price', 'buy_date', 'stock_num']]
    else:
        data_result_not_sell = None

    return data_result_sell, data_result_not_sell, sell_money



def util_get_stock_value(data_hold, one_date):

    if data_hold is None:
        return 0


    if len(data_hold) == 0:
        return 0

    else:
        list_code = data_hold['code'].unique().tolist()
        stock_price = \
        get_price(list_code, count=1, end_date=one_date, frequency='daily', fields=['close'], panel=False)[
            ['code', 'close']]

        stock_price = pd.merge(data_hold,stock_price,on=['code'])

        stock_price['money'] = stock_price.apply(
            lambda x: x['close'] * x['stock_num'], axis=1)

        return stock_price['money'].sum()


def util_middle_high_weight(num):
    num_left = num - int(num / 2)
    a1 = np.linspace(1, 10, num / 2)
    a2 = np.linspace(10, 1, num_left)
    a = np.append(a1, a2)
    theta = 1 / a.sum()
    b = theta * a
    return b.tolist()


def util_line_weight(num):
    a = np.linspace(1, 10, num)
    theta = 1.0 / sum(a)
    b = a * theta
    return b.tolist()


def util_get_stockCount_weight(pd_alpha,list_code,end_date):
    pd_alpha = pd_alpha[pd_alpha['end_date']==end_date]
    pd_alpha = pd_alpha[pd_alpha['code'].map(lambda x: x in list_code)]

    def weight_cal(x):
        if 40<x<80:
            return 3
        elif x>20 or x<100:
            return 2
        else:
            return 1

    if len(pd_alpha)==0:
        return None

    pd_alpha['weight'] = pd_alpha['alpha_stockCount'].map(weight_cal)
    weight_sum = pd_alpha['weight'].sum()
    pd_alpha['weight'] = pd_alpha['weight']/weight_sum
    return pd_alpha[['code','weight']]


def util_get_pct_stockCount_weight(pd_alpha,list_code,end_date):
    pd_alpha = pd_alpha[pd_alpha['end_date']==end_date]
    pd_alpha = pd_alpha[pd_alpha['code'].map(lambda x: x in list_code)]

    def weight_cal(x):
        if x<0.01:
            return 1
        elif x<0.1:
            return 2
        elif x<0.5:
            return 3
        elif x<0.7:
            return 2
        else:
            return 1

    if len(pd_alpha)==0:
        return None

    pd_alpha['weight'] = pd_alpha['alpha_pct_stockCount'].map(weight_cal)
    weight_sum = pd_alpha['weight'].sum()
    pd_alpha['weight'] = pd_alpha['weight']/weight_sum
    return pd_alpha[['code','weight']]

def util_get_profit_month_weight(pd_alpha,list_code,end_date):
    pd_alpha = pd_alpha[pd_alpha['end_date']==end_date]
    pd_alpha = pd_alpha[pd_alpha['code'].map(lambda x: x in list_code)]

    def weight_cal(x):
        if -0.05<x<0.05:
            return 3
        elif x<0.1:
            return 2
        elif x>-0.1:
            return 2
        else:
            return 1

    if len(pd_alpha)==0:
        return None

    pd_alpha['weight'] = pd_alpha['alpha_profit_month_3'].map(weight_cal)
    weight_sum = pd_alpha['weight'].sum()
    pd_alpha['weight'] = pd_alpha['weight']/weight_sum
    return pd_alpha[['code','weight']]


def last_day_of_month(any_day):
    import datetime
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def buy_every_fund_released(list_year, to_str=True):
    import datetime
    result = []
    for year in list_year:
        for month in [1, 4, 7, 10]:
            day = last_day_of_month(datetime.date(year, month, 1))
            if to_str:
                result.append(day.strftime('%Y-%m-%d'))
            else:
                result.append(day)
    return result


def buy_every_month_end(list_year, to_str=True):
    import datetime
    result = []
    for year in list_year:
        for month in range(1,13):
            day = last_day_of_month(datetime.date(year, month, 1))
            if to_str:
                result.append(day.strftime('%Y-%m-%d'))
            else:
                result.append(day)
    return result



def util_maxDrawdown(return_list):
    i = np.argmax((np.maximum.accumulate(return_list) - return_list) / np.maximum.accumulate(return_list))  # 结束位置
    if i == 0:
        return 0
    j = np.argmax(return_list[:i])  # 开始位置
    return (return_list[j] - return_list[i]) / (return_list[j])




In [ ]:
# 基金持仓的基本数据，前缀为sn


def get_fund_portfolio_by_year(start_year, end_year):
    dict_fund_pub_date = {'第四季度': ('12-31', '01-31'), '年度': ('12-31', '03-31'), '第一季度': ('03-31', '04-30'),
                          '第二季度': ('06-30', '07-31'), '半年度': ('06-30', '08-31'), '第三季度': ('09-30', '10-31')}

    result = []

    for i in range(end_year - start_year + 1):
        year_iter = start_year + i

        for key, value in dict_fund_pub_date.items():

            if key=='第四季度' or key=='年度':
                year_real_start = str(year_iter-1) + '-' + value[0]
            else:
                year_real_start = str(year_iter) + '-' + value[0]

            year_real_end = str(year_iter) + '-' + value[1]

            print(year_real_start,year_real_end)

            if datetime.strptime(year_real_end,'%Y-%m-%d')>datetime.today():
                break

            q = query(finance.FUND_PORTFOLIO.code, finance.FUND_PORTFOLIO.total_asset).filter(
                finance.FUND_PORTFOLIO.pub_date <= year_real_end,
                finance.FUND_PORTFOLIO.pub_date >= year_real_start)
            fund_asset = finance.run_query(q)
            # 筛选出净值大约1个亿的基金，基金净值过小容易被清盘，风格漂移
            l_fund_asset_code = fund_asset[fund_asset['total_asset'] >= 100000000]['code'].values.tolist()

            total_fund = len(set(l_fund_asset_code))

            # 得到符合要求的基金的所有持仓，年报所有股票，季报为持仓TOP10的
            fund_portfolio = util_get_fund_portfolio_by_chunks(l_fund_asset_code,year_real_start,year_real_end,key)
            fund_portfolio['total_fund'] = total_fund
            result.append(fund_portfolio)


    pd_result = pd.concat(result)
    return pd_result



sn_fund_portfolio = get_fund_portfolio_by_year(2014, 2021)
sn_fund_portfolio = util_normalize_code(sn_fund_portfolio,'symbol')
sn_fund_portfolio = util_add_name(sn_fund_portfolio)
sn_fund_portfolio = util_add_pub_year(sn_fund_portfolio)
sn_fund_portfolio[['symbol','code','display_name','pub_date','start_date','end_date','report_type','stock_count','total_fund','share_sum','market_cap_sum']].to_excel('data/基金持仓数据2014-2021.xlsx')

del sn_fund_portfolio



In [ ]:
# 前缀为ft
# Fund_TurnOverRatio 因子有3个目的，所有的因子围绕这3个目的展开
# 1，找到基金经理在大量建仓的标的物，前缀为 alpha_ft_1 或者 condition_ft_1
# 2，找到大量建仓的标的物集中建仓的时间，前缀为 alpha_ft_2 或者 condition_ft_2
# 3，找到目前标的物仓位和实仓看多满仓的差距，前缀为 alpha_ft_3 或者 condition_ft_3

# 前提：标的物上市超过一年，市场价格能反映其真实价值，前缀为 condition_ft_0


# @todo: 考虑下基金本身每年数量的增加情况，新基金



def ft_pre(fund_portfolio):
    fund_portfolio = pd.read_excel('data/基金持仓数据2014-2021.xlsx')
    fund_portfolio = fund_portfolio[
        (fund_portfolio['display_name'] != '未知') & (fund_portfolio['pub_date'] != '未知')]
    fund_portfolio = fund_portfolio.sort_values(
        by=['code', 'display_name', 'start_date', 'end_date', 'report_type', 'stock_count']).drop_duplicates(
        subset=['code', 'display_name', 'start_date', 'end_date', 'report_type'], keep='last').drop(['symbol'], axis=1)
    fund_portfolio['pub_date'] = fund_portfolio['pub_date'].map(
        lambda x: x[0:10] if type(x) == str else x.strftime('%Y-%m-%d'))
    fund_portfolio['stock_count'] = fund_portfolio['stock_count'].map(lambda x: 10 if x < 10 else x)
    fund_portfolio.sort_values(by=['code', 'start_date', 'end_date']).to_csv('data/基金持仓数据2014-2021_new.csv')
    print('数据预处理 Done')

# ###数据预处理
# ft_pre(data_all)






def condition_ft_0_1(data_o, pub_year=1):
    """
    股票上市相对于起始日期超过一年,仅仅看季报
    :param pub_year:
    :param data_o:
    :return:
    """
#     data_o['start_date'] = data_o['start_date'].map(lambda x:datetime.strptime(x, '%Y-%m-%d'))
#     data_o['pub_date'] = data_o['pub_date'].map(lambda x:datetime.strptime(x, '%Y-%m-%d'))
#     data_o['end_date'] = data_o['end_date'].map(lambda x:datetime.strptime(x, '%Y-%m-%d'))

    data_o['pub_date'] = data_o['pub_date'].map(lambda x: x[0:10] if type(x) == str else x.strftime('%Y-%m-%d'))


    data_o = data_o[
        data_o.apply(
            lambda x: (datetime.strptime(x['start_date'], '%Y-%m-%d') - datetime.strptime(x['pub_date'], '%Y-%m-%d')).days > pub_year * 365,
            axis=1)]
    data_o = data_o[data_o['report_type'].map(lambda x: '季度' in x)]
    print('fund_condition_1 done')
    return data_o


def alpha_ft_1_1(data_o, m=0.01):
    """
    基金持有股票的绝对数量以及其变化
    :param m:
    :param data_o:
    :return:
    """
    data_o = data_o.sort_values(by=['code', 'display_name', 'start_date', 'end_date', 'report_type'])
    data_o['pct_stock_count'] = data_o.groupby(['code'])['stock_count'].pct_change()
    print('alpha_ft_1_1 done')
    return data_o[['code', 'display_name','end_date','stock_count','pct_stock_count']]


def alpha_ft_1_3(data_o, m=0.01):
    """
    基金持有股票的相对数量，以及其变化
    :param m:
    :param data_o:
    :return:
    """
    data_o = data_o.sort_values(by=['code', 'display_name', 'start_date', 'end_date', 'report_type'])
    data_o['ratio_stock_count'] = data_o['stock_count']/data_o['total_fund']
    data_o['pct_ratio_stock_count'] = data_o.groupby(['code'])['ratio_stock_count'].pct_change()
    print('alpha_ft_1_3 done')
    return data_o[['code', 'display_name','end_date','ratio_stock_count','pct_ratio_stock_count']]


def alpha_ft_1_2(data_o):
    """
    某一股票被基金持有的市值/股票总流量市值 的变化相较于上一个季度的占比
    :param data_o:
    :return:
    """
    list_end_date = set(data_o['end_date'].values.tolist())
    result_cap = []
    for one_end_date in list_end_date:
        list_code = data_o[data_o['end_date']==one_end_date]['code'].unique().tolist()
        r = get_valuation(list_code, end_date=one_end_date, count=1,fields=['circulating_cap'])
        r['end_date'] = one_end_date
        del r['day']
        result_cap.append(r)
    pd_cap_result = pd.concat(result_cap,ignore_index = True)
    pd_cap_result = pd.merge(data_o,pd_cap_result,on = ['code','end_date'])
    pd_cap_result['ratio_share_sum'] = pd_cap_result['share_sum']/pd_cap_result['circulating_cap']
    pd_cap_result['pct_ratio_share_sum'] = pd_cap_result['ratio_share_sum'].pct_change()
    print('alpha_ft_1_2 done')
    return pd_cap_result[['code', 'display_name','end_date','share_sum','circulating_cap','ratio_share_sum','pct_ratio_share_sum']]



def alpha_ft_2_1_and_2(data_o):
    """
    某一股票被基金持有的市值/股票总流量市值 的变化相较于上一个季度的占比
    :param data_o:
    :return:
    """

    def agg_func(x,windows):
        x_values = x.values
        result = []
        for window in windows:
            x = x_values[-window:].mean()
            result.append(x)
        return result


    list_end_date = set(data_o['end_date'].values.tolist())
    result_cap = []

    my_windows = [3,7,30,90]
    columns_name = ['turnOver_window_'+str(win) for win in my_windows]

    for one_end_date in list_end_date:
        list_code = data_o[data_o['end_date']==one_end_date]['code'].unique().tolist()
        result_tmp = util_run_func_by_chunk(list_code,100,get_valuation,end_date=one_end_date, count=90,fields=['turnover_ratio'])
        pd_result_tmp = pd.concat(result_tmp,ignore_index = True).sort_values(by = ['code','day'])

        pd_result_tmp = pd_result_tmp.groupby(['code'])['turnover_ratio'].agg(lambda x: agg_func(x,my_windows)).reset_index()

        pd_result_tmp[columns_name] = pd.DataFrame(pd_result_tmp['turnover_ratio'].values.tolist(), columns=columns_name)
        del pd_result_tmp['turnover_ratio']

        pd_result_tmp['end_date'] = one_end_date
        result_cap.append(pd_result_tmp)

    pd_cap_result = pd.concat(result_cap,ignore_index = True)
    pd_cap_result = pd.merge(data_o,pd_cap_result,on = ['code','end_date'])
    return_col = ['code', 'display_name','end_date']+columns_name
    print('alpha_ft_2_1_and_2 done')
    return pd_cap_result[return_col]



def get_ft_result(path_final):
    data_all = pd.read_csv('data/基金持仓数据2014-2021_new.csv')
    ft_result = condition_ft_0_1(data_all)
    ft_alpha_ft_1_1 = alpha_ft_1_1(ft_result)
    ft_alpha_ft_1_2 = alpha_ft_1_2(ft_result)
    ft_alpha_ft_1_3 = alpha_ft_1_3(ft_result)
    ft_alpha_ft_2_1_and_2 = alpha_ft_2_1_and_2(ft_result)
    result = pd.merge(ft_alpha_ft_1_1,ft_alpha_ft_1_2,on=['code', 'display_name','end_date'])
    result = pd.merge(result,ft_alpha_ft_1_3,on=['code', 'display_name','end_date'])
    result = pd.merge(result,ft_alpha_ft_2_1_and_2,on=['code', 'display_name','end_date'])
    util_drop_unnanme_col(result).to_excel(path_final)


get_ft_result('result/Fund_TurnOverRatio.xlsx')
ft_data = pd.read_excel('result/Fund_TurnOverRatio.xlsx')
get_pd_K_raise_3m_later(ft_data).to_excel('result/Fund_TurnOverRatio_3m_revenue.xlsx')



In [ ]:
# 基础数据准备


# pd_fund_alpha = pd.read_excel('result/Fund_TurnOverRatio_3m_revenue.xlsx')
# pd_price_all = util_get_month_K(pd_fund_alpha['code'].unique().tolist())
# pd_paused = util_get_month_paused(pd_fund_alpha['code'].unique().tolist())
# pd_turnOver = util_get_month_turnOver(pd_fund_alpha['code'].unique().tolist())
# pd_price_all.to_csv('data/price_month.csv')
# del pd_price_all
# pd_paused.to_csv('data/paused_month.csv')
# del pd_paused
# pd_turnOver.to_csv('data/turnOver_mean_month.csv')

pd_fund_alpha = pd.read_excel('result/Fund_TurnOverRatio_3m_revenue.xlsx')
pd_price_all = pd.read_csv('data/price_month.csv')
pd_paused = pd.read_csv('data/paused_month.csv')
pd_turnOver = pd.read_csv('data/turnOver_mean_month.csv')


pd_data_all = pd.merge(pd_price_all,pd_paused,on = ['code','end_date'],how='outer')
pd_data_all = pd.merge(pd_data_all,pd_turnOver,on = ['code','end_date'],how='outer')
pd_data_all = pd.merge(pd_data_all,pd_fund_alpha,on = ['code','end_date'],how='outer')





# def plus_10_func(data_o):
#     x = data_o['stock_count'].values
#     result = []
#     for one_x in x:
#         if len(result)==0:
#             last_value = 0
#         else:
#             last_value = result[-1]
#         if one_x>10:
#             result.append(last_value+1)
#         else:
#             result.append(last_value)
#     return pd.Series(result,index=data_o['end_date'])

# pd_fund_alpha['plus_10'] = pd_fund_alpha.groupby(['code']).apply(plus_10_func).values
# pd_fund_alpha_tmp = pd_fund_alpha[pd_fund_alpha['plus_10']>=3]


# def get_alpht_month_turnOver(data_o):
#     list_code = data_o['code'].unique().tolist()
#     pd_turnOver = util_get_month_turnOver(list_code)
#     pd_result = pd.merge(pd_turnOver,data_o,how='left',on=['code','end_date'])
#     return pd_result

# data_origin = get_alpht_month_turnOver(pd_fund_alpha)

# data_origin_2 = pd.merge(data_origin,pd_paused,how='left',on=['code','end_date'])
# data_origin_2['cum_paused'] = data_origin_2.groupby(['code'])['paused'].cumsum()
# data_origin_2 = pd.merge(data_origin_2,pd_price_all,on=['code','end_date'])

del pd_price_all
del pd_paused
del pd_turnOver
del pd_fund_alpha
print('done')


In [ ]:
# 因子构建

def alpha_turnOver_30(data_o, N):
    def alpha_turnOver_30_inner(data_grp_o):
        threshold = 0
        x = data_grp_o['turnOver_monthly'].values
        result = []
        for i in range(len(x)):
            one_x = x[i]
            if threshold >= N:
                result.append(one_x)
            else:
                result.append(0)
            if 0.3 < one_x < 0.8:
                threshold += 1
            else:
                threshold = 0
        return pd.Series(result, index=data_grp_o['end_date'])

    pd_tmp = data_o.groupby(['code']).apply(alpha_turnOver_30_inner).unstack().reset_index().rename(
        columns={0: 'alpha_turnOver_30'}).set_index(['code', 'end_date'])
    r = data_o.set_index(['code', 'end_date'])
    r = pd.concat([r, pd_tmp], axis=1).reset_index()
    r = r[r['alpha_turnOver_30'] > 0]
    r = r[['end_date', 'code', 'alpha_turnOver_30']].sort_values(
        by=['end_date', 'alpha_turnOver_30']).set_index(['end_date', 'code'])
    return r


def alpha_pct_stockCount(data_o):
    r = data_o[['end_date', 'code', 'pct_stock_count']].sort_values(by=['code','end_date', 'pct_stock_count'])
    r = r.fillna(0).rename(columns={'pct_stock_count': 'alpha_pct_stockCount'})
    r = r.sort_values(by=['end_date', 'alpha_pct_stockCount']).set_index(['end_date', 'code'])
    return r


def alpha_stockCount(data_o):
    r = data_o[['end_date', 'code', 'stock_count']].sort_values(by=['code','end_date', 'stock_count'])
    r = r.fillna(0).rename(columns={'stock_count': 'alpha_stockCount'})
    r = r.sort_values(by=['end_date', 'alpha_stockCount']).set_index(['end_date', 'code'])
    return r



def alpha_paused(data_o):
    r = data_o[data_o['paused'] < 1][['end_date', 'code', 'paused']].sort_values(by=['end_date', 'paused'])
    r = r.rename(columns={'paused': 'alpha_paused'}).drop_duplicates().set_index(['end_date', 'code'])
    return r


def alpha_profit_month(data_p):
    def alpha_profit_month_inner(data_grp_o):
        price_all = data_grp_o['close'].values
        cum_profit_10 = []
        cum_profit_1 = []
        cum_profit_3 = []
        for i, one_price in enumerate(price_all):
            if i >= 1:
                cum_profit_1.append((one_price - price_all[0]) / price_all[0])
            else:
                cum_profit_1.append(0)
            if i >= 3:
                cum_profit_3.append(((one_price - price_all[i - 3]) / price_all[i - 3]))
            else:
                cum_profit_3.append(0)
            if i >= 10:
                cum_profit_10.append((one_price - price_all[i - 10]) / price_all[i - 10])
            else:
                cum_profit_10.append(0)

        pd_result = pd.DataFrame.from_dict(
            {'cum_profit_10': cum_profit_10, 'cum_profit_1': cum_profit_1, 'cum_profit_3': cum_profit_3})
        pd_result['smart_price'] = 0.3 * pd_result['cum_profit_10'] + 0.3 * pd_result['cum_profit_3'] + 0.4 * pd_result[
            'cum_profit_1']
        pd_result = pd_result.set_index(data_grp_o['end_date'])
        return pd_result

    r = data_p.groupby(['code']).apply(alpha_profit_month_inner).reset_index().drop_duplicates().set_index(['end_date', 'code'])
    r = r.rename(columns={'cum_profit_10': 'alpha_profit_month_10','cum_profit_1':'alpha_profit_month_1','cum_profit_3':'alpha_profit_month_3'})
    return r


def alpha_plus_10(data_o,N=3):
    def alpha_plus_10_inner(data_grp_o):
        x = data_grp_o['stock_count'].values
        result = []
        for one_x in x:
            if len(result) == 0:
                last_value = 0
            else:
                last_value = result[-1]
            if one_x > 10:
                result.append(last_value + 1)
            else:
                result.append(last_value)
        return pd.Series(result, index=data_grp_o['end_date'])

    pd_tmp = data_o.fillna(0).groupby(['code']).apply(alpha_plus_10_inner).unstack().reset_index().rename(
        columns={0: 'alpha_plus_10'}).set_index(['code', 'end_date'])
    r = data_o.set_index(['code', 'end_date'])
    r = pd.concat([r, pd_tmp], axis=1).reset_index()
    r = r[r['alpha_plus_10'] >= N][['end_date','code','alpha_plus_10']].drop_duplicates().set_index(['end_date', 'code'])
    return r


def main_alpha():
    alpha_chosen = {
        'alpha_turnOver_30': {'N': 3},
        'alpha_pct_stockCount': {},
        'alpha_paused': {},
        'alpha_profit_month': {},
        'alpha_plus_10': {'N': 4},
        'alpha_stockCount':{}
    }

    result = []
    for one_m, one_k in alpha_chosen.items():
        r_tmp = globals().get(one_m)(pd_data_all,**one_k)
        result.append(r_tmp)

    r_all = pd.concat(result, axis=1).reset_index()
    return r_all


alpha_concat = main_alpha()
print('done')

In [ ]:
# 购买策略：
# 择时：每月末？每基金财年末
# 选量：均等资金？均等量
# 规则：依照什么排序
# 候选集：对那些标的进行排序



def buy_method_market(data_all,method_name,**kargs):

    def buy_by_sort(**kargs_inner):
        data_o = data_all.dropna()
        if kargs.get('sort') is not None:
            data_o = data_o.sort_values(**kargs.get('sort')).groupby(['end_date']).head(kargs_inner.get('N'))
        else:
            data_o = data_o.sample(frac=1).groupby(['end_date']).head(kargs_inner.get('N'))
        buy_time = MarketTiming.buy_every_fund_released([2013,2014, 2015, 2016,2017,2018,2019,2020,2021])
        data_o = pd.merge(data_o,buy_time,on = ['end_date'])
        return data_o


    def buy_custom_1(**kargs_inner):
        data_o = data_all.dropna()
        fund_release_date = buy_every_fund_released([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])
        month_end = buy_every_month_end([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])
        list_end_date = []
        list_code = []
        code_bought = set()
        flag = 1

        for one_date in month_end:
            data_one_date = data_o[data_o['end_date'] == one_date]
            if len(data_one_date) == 0:
                continue
            if one_date in fund_release_date:
                data_one_date = data_one_date[data_one_date['alpha_stockCount']>10]
                data_one_date = data_one_date[data_one_date['alpha_pct_stockCount'] >= kargs_inner.get('alpha_psc_thold_bot')]
                data_one_date = data_one_date[data_one_date['alpha_pct_stockCount'] <= kargs_inner.get('alpha_psc_thold_top')]
                code_now = data_one_date.sort_values(**kargs.get('sort_1')).head(kargs_inner.get('N_1'))[
                    'code'].values.tolist()
                flag = 2
            else:
                data_one_date = data_one_date[data_one_date['code'].map(lambda x: x in code_bought)]
                if flag == 2:
                    code_now = data_one_date.sort_values(**kargs.get('sort_2')).head(kargs_inner.get('N_2'))[
                        'code'].values.tolist()
                    flag += 1
                elif flag == 3:
                    code_now = data_one_date.sort_values(**kargs.get('sort_3')).head(kargs_inner.get('N_3'))[
                        'code'].values.tolist()
                else:
                    continue
            list_end_date += [one_date for i in range(len(code_now))]
            list_code += code_now
            code_bought = set(code_now)
        return pd.DataFrame({'end_date': list_end_date, 'code': list_code})


    def buy_custom_2(**kargs_inner):
        data_o = data_all.dropna()
        fund_release_date = buy_every_fund_released([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])
        month_end = buy_every_month_end([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])
        list_end_date = []
        list_code = []
        code_bought = set()
        flag = 1

        for one_date in month_end:
            data_one_date = data_o[data_o['end_date'] == one_date]
            if len(data_one_date) == 0:
                continue
            if one_date in fund_release_date:
                data_one_date = data_one_date[data_one_date['alpha_stockCount']>10]
                data_one_date = data_one_date[data_one_date['alpha_pct_stockCount'] >= kargs_inner.get('alpha_psc_thold_bot')]
                data_one_date = data_one_date[data_one_date['alpha_pct_stockCount'] <= kargs_inner.get('alpha_psc_thold_top')]
                code_now = data_one_date.sort_values(**kargs.get('sort_1')).head(kargs_inner.get('N_1'))[
                    'code'].values.tolist()
                flag = 2
            else:
                data_one_date = data_one_date[data_one_date['code'].map(lambda x: x in code_bought)]
                if flag == 2:
                    code_now = data_one_date.sort_values(**kargs.get('sort_2')).head(kargs_inner.get('N_2'))[
                        'code'].values.tolist()
                    flag += 1
                elif flag == 3:
                    code_now = data_one_date.sort_values(**kargs.get('sort_3')).head(kargs_inner.get('N_3'))[
                        'code'].values.tolist()
                else:
                    continue
            list_end_date += [one_date for i in range(len(code_now))]
            list_code += code_now
            code_bought = set(code_now)
        return pd.DataFrame({'end_date': list_end_date, 'code': list_code})




    return locals().get(method_name)(**kargs)


def format_buy(data_o):
    date_all = buy_every_month_end([2013,2014, 2015, 2016,2017,2018,2019,2020,2021])
    portfolio_last = set()
    portfolio_now = set()
    list_opo = []
    list_code = []
    list_date = []
    for one_date in date_all:
        satisfy_code = data_o[data_o['end_date'] == one_date]['code'].unique()
        for one_code in satisfy_code:
            list_date.append(one_date)
            list_code.append(one_code)
            if one_code in portfolio_last:
                list_opo.append('hold')
                portfolio_now.add(one_code)
            else:
                list_opo.append('buy')
                portfolio_now.add(one_code)
        for one_code in (portfolio_last - portfolio_now):
            list_date.append(one_date)
            list_code.append(one_code)
            list_opo.append('sell')
        portfolio_last = portfolio_now.copy()
        portfolio_now = set()

    r1 = pd.DataFrame.from_dict({'end_date': list_date, 'code': list_code, 'operation': list_opo})
    return r1


def main_buy():

    # #### base
#     rank_method = {'N': 4}
#     r_tmp = buy_method_market(alpha_concat, 'buy_by_sort', **rank_method)


    # #### 方案一
#     rank_method = {'sort': {'by': ['alpha_pct_stockCount', 'alpha_profit_month_1', 'alpha_turnOver_30'],
#                             'ascending': [False, False, True]}, 'N': 4}
#     r_tmp = buy_method_market(alpha_concat, 'buy_by_sort', **rank_method)

#     # 方案二
#     rank_method = {'sort_1': {'by': ['alpha_pct_stockCount', 'alpha_profit_month_3', 'alpha_turnOver_30'],
#                               'ascending': [False, False, True]}, 'N_1': 10,
#                    'sort_2': {'by': ['alpha_profit_month_3', 'alpha_turnOver_30'],
#                               'ascending': [True, True]}, 'N_2': 10,
#                    'sort_3': {'by': ['alpha_profit_month_3', 'alpha_turnOver_30'],
#                               'ascending': [True, True]}, 'N_3': 10,'alpha_pct_stockCount_thold':0.05}
#     r_tmp = buy_method_market(alpha_concat, 'buy_custom_1', **rank_method)


    # 方案三
    rank_method = {'sort_1': {'by': ['alpha_pct_stockCount', 'alpha_profit_month_3', 'alpha_turnOver_30'],
                              'ascending': [False, False, True]}, 'N_1': 10,
                   'sort_2': {'by': ['alpha_profit_month_3', 'alpha_turnOver_30'],
                              'ascending': [True, True]}, 'N_2': 10,
                   'sort_3': {'by': ['alpha_profit_month_3', 'alpha_turnOver_30'],
                              'ascending': [True, True]}, 'N_3': 10,'alpha_psc_thold_bot':-0.03,'alpha_psc_thold_top':1000000}
    r_tmp = buy_method_market(alpha_concat, 'buy_custom_1', **rank_method)

    # 方案四
#     rank_method = {'sort_1': {'by': ['alpha_pct_stockCount', 'alpha_profit_month_3', 'alpha_turnOver_30'],
#                               'ascending': [False, False, True]}, 'N_1': 10,
#                    'sort_2': {'by': ['alpha_profit_month_3', 'alpha_turnOver_30'],
#                               'ascending': [True, True]}, 'N_2': 10,
#                    'sort_3': {'by': ['alpha_profit_month_3', 'alpha_turnOver_30'],
#                               'ascending': [True, True]}, 'N_3': 10,'alpha_psc_thold_bot':0.02,'alpha_psc_thold_top':1000000}
#     r_tmp = buy_method_market(alpha_concat, 'buy_custom_1', **rank_method)



    r_tmp = format_buy(r_tmp)
    return r_tmp



buy_result = main_buy()
print('done')

In [ ]:

def back_analyse(data_all):

    year_list = [2014,2015,2016,2017,2018,2019,2020,2021]

    stock_bought = {}
    date_iter = buy_every_month_end(year_list)
    date_fund_release =  buy_every_fund_released(year_list)
    data_hold = pd.DataFrame.from_dict({'code':[],'buy_price':[],'end_date':[],'stock_num':[]})
    result = []
    result_month_stock = []
    result_hs = []
    money_available = 1000000
    all_value = 1000000
    data_all = data_all[data_all['operation']!='sell']
    for i,one_date in enumerate(date_iter):
        pd_now = data_all[data_all['end_date']==one_date]
        code_for_buy = pd_now['code'].unique().tolist()
        data_for_sell = data_hold[data_hold['code'].map(lambda x:x not in code_for_buy)]
        pd_stock_sell,pd_stock_not_sell,sell_money = util_sell_stocks(data_for_sell,one_date)
        if pd_stock_sell is not None:
            result.append(pd_stock_sell)
        money_available+=sell_money

        data_still_hold = data_hold[data_hold['code'].map(lambda x:x in code_for_buy)]
        value_stock = util_get_stock_value(data_still_hold,one_date)

        buy_length = len(code_for_buy)
        if one_date in date_fund_release:
#             pd_weight = util_get_pct_stockCount_weight(alpha_concat,code_for_buy,one_date)
            buy_now,money_buy = util_buy_stocks(value_stock+money_available,one_date,code_for_buy,util_line_weight(buy_length))
        else:
#             pd_weight = util_get_profit_month_weight(alpha_concat,code_for_buy,one_date)
            buy_now,money_buy = util_buy_stocks(value_stock+money_available,one_date,code_for_buy,util_line_weight(buy_length))

        if (buy_now is not None) or (pd_stock_not_sell is not None) :
            data_hold = pd.concat([buy_now,pd_stock_not_sell],sort=True,ignore_index=True)
            result_month_stock.append(data_hold)
        else:
            data_hold = pd.DataFrame.from_dict({'code':[],'buy_date':[],'buy_price':[],'stock_num':[]})

        money_available=value_stock+money_available - money_buy
        value_stock = util_get_stock_value(data_hold,one_date)

        tmp_d = pd.DataFrame.from_dict({'end_date':[one_date],'cash':[money_available],'value_stock':[value_stock],'value_all':[money_available+value_stock]})
        result_hs.append(tmp_d)

    pd_result = pd.concat(result)
    pd_result_month_stock = pd.concat(result_month_stock)
    pd_result_hs = pd.concat(result_hs)
#     pd_result.to_excel('result/r_pd_result.xlsx')
#     pd_result_month_stock.to_excel('result/r_pd_result_month_stock.xlsx')
#     pd_result_hs.to_excel('result/r_pd_result_hs.xlsx')

    draw_back = util_maxDrawdown(pd_result_hs['value_all'].values.tolist())
    print('总收益：',(pd_result_hs['value_all'].values[-1] - pd_result_hs['value_all'].values[0])/pd_result_hs['value_all'].values[0])
    print('最大回撤：',draw_back)




back_analyse(buy_result)



